# Test Matching Insee/ETS

## INSEE

- https://s3.console.aws.amazon.com/s3/object/calfdata/INSEE/Stock/ETS/
        - INSEE/Stock/ETS/StockEtablissement_utf8.csv
        
```
['siren', 'siret']
```

## INPI

- https://s3.console.aws.amazon.com/s3/buckets/calfdata/INPI/TC_1/Stock_processed/
    - INPI/TC_1/Stock_processed/initial_ETS.gz
    - INPI/TC_1/Stock_processed/initial_ETS.json
    
Colonnes test:

```
["Siren","Date_Immatriculation", "Date_Clôture", "Date_Greffe"]
```

## Sauvegarde

* La liste des SIREN matchés sera sauvegardée selon leur nature et origine
  * nature → ACTES/COMPTES/ETS/etc
  * origine → initial/partiel/new/evt

Les matchés seront sauvegardé dans calfdata/SIRETISATION/matche/ au format suivant:

* insee_nature_origine_matche.gz
    * ex: insee_pm_initial_matche.gz
    
    

## Moteur de recherche TEST

* Insee
  * http://avis-situation-sirene.insee.fr/IdentificationListeSiret.action
* INPI/TC
  * https://data.inpi.fr/
* Infogreffe
  * https://www.infogreffe.fr/


Le siège ne donne pas de nouveau SIRET, il indique seulement le lieu de la juridiction

In [ ]:
import boto3, json
import dask.dataframe as dd
import pandas as pd
#import Match_inpi_insee.aws_connectors as aws
#from tqdm.notebook import tqdm
#import tqdm
%load_ext autoreload
%autoreload 2

In [ ]:
#instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
#bucket = 'calfdata'

In [ ]:
# instanciate AWS connection
#AWS_connection = aws.aws_instantiate(instance_aws, bucket)

### Créer fichier toutes les possibilités communes

In [ ]:
communes = pd.read_csv('temp_local\communes-01012019.csv').set_index('ncc').reindex(columns = ['nccenr', 'libelle'])#.unstack()
communes.loc[lambda x: x['libelle'].isin(['Châtillon-sur-Chalaronne'])]

In [ ]:
communes = (pd.read_csv('temp_local\communes-01012019.csv')
            .set_index('ncc')
            .reindex(columns=['nccenr', 'libelle'])
            .assign(
    noaccent=lambda x: x['nccenr'].str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8'),
    nccenr_noponc=lambda x: x['nccenr'].str.replace('[^\w\s]', ' '),
    libelle_noponc=lambda x: x['libelle'].str.replace('[^\w\s]', ' '),
    noaccent_noponc=lambda x: x['noaccent'].str.replace('[^\w\s]', ' '),
    uppercase=lambda x: x.index,
    nccenr_uppercase=lambda x: x['nccenr'].str.upper(),
    libelle_uppercase=lambda x: x['libelle'].str.upper(),
    noaccent_uppercase=lambda x: x['noaccent'].str.upper(),
    nccenr_noponc_uppercase=lambda x: x['nccenr_noponc'].str.upper(),
    libelle_noponc_uppercase=lambda x: x['libelle_noponc'].str.upper(),
    noaccent_noponc_uppercase=lambda x: x['noaccent_noponc'].str.upper(),
    nccenr_lowercase=lambda x: x['nccenr'].str.lower(),
    libelle_lowercase=lambda x: x['libelle'].str.lower(),
    noaccent_lowercase=lambda x: x['noaccent'].str.lower(),
    nccenr_noponc_lowercase=lambda x: x['nccenr_noponc'].str.lower(),
    libelle_noponc_lowercase=lambda x: x['libelle_noponc'].str.lower(),
    noaccent_noponc_lowercase=lambda x: x['noaccent_noponc'].str.lower(),
    nccenr_noarrond1=lambda x: x['nccenr'].str.replace(
        'er Arrondissement', ''),
    uppercase_noarrond1=lambda x: x['uppercase'].str.replace(
        'ER ARRONDISSEMENT', ''),
    lowercase_noarrond1=lambda x: x['nccenr_lowercase'].str.replace(
        'er arrondissement', ''),
    nccenr_noarrond=lambda x: x['nccenr'].str.replace('e Arrondissement', ''),
    uppercase_noarrond=lambda x: x['uppercase'].str.replace(
        'E ARRONDISSEMENT', ''),
    lowercase_noarrond=lambda x: x['nccenr_lowercase'].str.replace(
        'e arrondissement', ''),
)
)

for n in communes.columns:
    var_ = '{}_ST'.format(n)
    var_1 = '{}_st'.format(n)
    var_2 = '{}_St'.format(n)
    
    communes[var_] = communes[n].str.replace('SAINT', 'ST')
    communes[var_1] = communes[n].str.replace('Saint', 'st')
    communes[var_2] = communes[n].str.replace('Saint', 'St')
    
    var_ = '{}_Sbar'.format(n)
    var_1 = '{}_sbar'.format(n)
    
    communes[var_] = communes[n].str.replace('SUR', 'S/')
    communes[var_1] = communes[n].str.replace('sur', 's/')
    
communes = (communes
            .stack()
            .rename('possibilite')
            .reset_index()
            .drop(columns='level_1')
            .drop_duplicates(subset=['possibilite']))
communes.head()

## Valeurs communes dans INSEE/INPI ETS

| INSEE                     | INPI         |                         |
|---------------------------|--------------|-------------------------|
| Var                       | Var          | comment                 |
| siren                     | Siren        |                         |
| codePostalEtablissement   | Code_Postal  | Si Type == 'principale' |
| codePostal2Etablissement  | Code_Postal  | Si Type == 'secondaire' |
| codeCommuneEtablissement  | Code_Commune | Si Type == 'principale' |
| codeCommune2Etablissement | Code_Commune | Si Type == 'secondaire' |
|                           |              |                         |
|                           |              |                         |
|                           |              |                         |

In [ ]:
#insee = AWS_connection.url_instance_bucket(path_file = 'INSEE/Stock/ETS/StockEtablissement_utf8.csv')
#ets = AWS_connection.url_instance_bucket(path_file = 'INPI/TC_1/Stock_processed/initial_ETS.gz')
#ets

In [ ]:
insee = r"\temp_local\StockEtablissement_utf8.csv"
ets = r"\temp_local\initial_ETS.gz"

## Matching établissement principal

Ici, on filtre les variables communes pour l'INSEE & INPI établissements secondaires.

### Candidats

**INSEE**

https://www.sirene.fr/sirene/public/static/liste-variables

- numeroVoieEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/numeroVoieEtablissement
- indiceRepetitionEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/indiceRepetitionEtablissement
- typeVoieEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/typeVoieEtablissement
- libelleVoieEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleVoieEtablissement
- complementAdresseEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/complementAdresseEtablissement
- codeCommuneEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codeCommuneEtablissement
- libelleCommuneEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleCommuneEtablissement
- codePostalEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codePostalEtablissement
- codeCedexEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codeCedexEtablissement
- libelleCedexEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleCedexEtablissement
- distributionSpecialeEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/distributionSpecialeEtablissement
- libelleCommuneEtrangerEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleCommuneEtrangerEtablissement
- codePaysEtrangerEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codePaysEtrangerEtablissement
- libellePaysEtrangerEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libellePaysEtrangerEtablissement

**INPI**

- Adresse_Ligne1/Adresse_Ligne2/Adresse_Ligne3: Selon les greffes, l’adresse (n°+ voie) sera présente soit en ligne1 adresse, soit en ligne2 adresse.
Toutes les lignes d’adresse ne sont pas nécessairement renseignées.


### Process

1) Step : Calculer le nombre de `nan` dans les colonnes de matching
2) Step : Compter le nombre de SIRET by SIREN
2) Step 2:  merge sur siren et code postal
3) Step 3:  merge sur siren et code commune

In [ ]:
# load data into dataframes
data_insee_ = dd.read_csv(insee,
                          usecols=['siren',
                                   'siret',
                                   "numeroVoieEtablissement",
                                   "indiceRepetitionEtablissement",
                                   "typeVoieEtablissement",
                                   "libelleVoieEtablissement",
                                   "complementAdresseEtablissement",
                                   "codeCommuneEtablissement",
                                   "libelleCommuneEtablissement",
                                   "codePostalEtablissement",
                                   "codeCedexEtablissement",
                                   "libelleCedexEtablissement",
                                   "distributionSpecialeEtablissement",
                                   "libelleCommuneEtrangerEtablissement",
                                   "codePaysEtrangerEtablissement",
                                   "libellePaysEtrangerEtablissement"
                                   ],
                          dtype={'siren': 'object',
                                 'siret': 'object',
                                 "numeroVoieEtablissement":'object',
                                   "indiceRepetitionEtablissement":'object',
                                   "typeVoieEtablissement":'object',
                                   "libelleVoieEtablissement":'object',
                                   "complementAdresseEtablissement":'object',
                                   "codeCommuneEtablissement":'object',
                                   "libelleCommuneEtablissement":'object',
                                   "codePostalEtablissement":'object',
                                   "codeCedexEtablissement":'object',
                                   "libelleCedexEtablissement":'object',
                                   "distributionSpecialeEtablissement":'object',
                                   "libelleCommuneEtrangerEtablissement":'object',
                                   "codePaysEtrangerEtablissement":'object',
                                   "libellePaysEtrangerEtablissement":'object'
                                 }
                          )

data_ets_ = (dd.read_csv(ets,
                         usecols=[
                             'Type',
                             'Siren',
                             'Code_Postal',
                             'Code_Commune',
                             'Adresse_Ligne1',
                             'Adresse_Ligne2',
                             'Adresse_Ligne3',
                             'Ville',
                             'Pays'
                         ],
                         dtype={
                             'Type': 'object',
                             'Siren': 'object',
                             'Code_Postal': 'object',
                             'Code_Commune': 'object',
                             'Adresse_Ligne1': 'object',
                             'Adresse_Ligne2': 'object',
                             'Adresse_Ligne3': 'object',
                             'Ville':'object',
                             'Pays':'object'
                         },
                         compression='gzip',
                         blocksize=None,
                         low_memory=False
                         )
             .compute()
             .rename(columns={"Siren": "siren"})
             .loc[lambda x: ~x['Type'].isin(['SIE'])]
             )

In [ ]:
data_ets_.shape

In [ ]:
siren_inpi = data_ets_['siren'].drop_duplicates()
len(siren_inpi)

In [ ]:
subset_insee = data_insee_.compute().loc[lambda x: x['siren'].isin(siren_inpi)]
subset_insee.shape

Siren INPI mais pas INSEE -> Cette entreprise a exercé son droit d'opposition auprès de l'INSEE. Ses données ne peuvent pas être diffusées publiquement.

In [ ]:
siren_to_remove = siren_inpi.loc[lambda x : ~x.isin(subset_insee['siren'])]
len(siren_to_remove)

In [ ]:
df_siren_to_find = data_ets_.loc[lambda x:
                                 (~x['siren'].isin(siren_to_remove))
                                 #| (~x['Adresse_Ligne1'].isin([np.nan]))
                                 #& (~x['Adresse_Ligne2'].isin([np.nan]))
                                 #& (~x['Adresse_Ligne3'].isin([np.nan]))
                                 #& (~x['Code_Postal'].isin([np.nan]))
                                 #& (~x['Ville'].isin([np.nan]))
                                 #& (~x['Code_Commune'].isin([np.nan]))
                                 ]
len(df_siren_to_find)

### Nan variables matching 

on exclue les variables avec que des nan dans les variables candidates

-> on les traitera après

In [ ]:
import numpy as np
siren_fullna = df_siren_to_find.loc[lambda x:
                      (x['Adresse_Ligne1'].isin([np.nan]))
                     & (x['Adresse_Ligne2'].isin([np.nan]))
                     & (x['Adresse_Ligne3'].isin([np.nan]))
                     & (x['Code_Postal'].isin([np.nan]))
                     & (x['Ville'].isin([np.nan]))
                     & (x['Code_Commune'].isin([np.nan]))
                     ]['siren']

In [ ]:
df_siren_to_find = df_siren_to_find.loc[lambda x:
                                 (~x['siren'].isin(siren_fullna))
                                 ]
len(siren_fullna)

In [ ]:
df_siren_to_find.shape

### Nombres d'ets par SIREN INSEE

On calcule le nombre d'etb pour le fichier INSEE.

In [ ]:
subset_insee_count = subset_insee.merge(
    (subset_insee
     .groupby('siren')['siren']
     .count()
     .rename('count')
     .reset_index())
)

In [ ]:
def siren_unique(df):
    """
    """
    print("Nombre total obs: {}".format(len(df)))
    count_ = (df
              .groupby('siren')['siren']
              .count()
              .rename('count')
              .reset_index()
              .groupby('count')['count']
              .count()
              .reset_index(name='total_count')
              .set_index('count')
              # .compute()
              .assign(pct=lambda x: x/x.sum())
              .iloc[:10, :]
              .style
              .format('{:,.2%}', subset=['pct'])
              )
    return count_

Quick stat

In [ ]:
siren_unique(df = subset_insee_count)

In [ ]:
df_siren_to_find.loc[lambda x: x['siren'].isin(['999990229'])]

In [ ]:
#(subset_insee_count
# .loc[lambda x: x['count'].isin([1])]
# .merge(
#     df_siren_to_find,
#     on='siren',
#     how='left',
#     indicator=True)
# )

## Missing values

In [ ]:
subset_insee_count.isna().sum().sort_values()

In [ ]:
df_siren_to_find.isna().sum().sort_values()

In [ ]:
df_siren_to_find.shape

In [ ]:
var_insee = ['siret',
             "numeroVoieEtablissement",
             "indiceRepetitionEtablissement",
             "typeVoieEtablissement",
             "libelleVoieEtablissement",
             "complementAdresseEtablissement",
             "codeCommuneEtablissement",
             "libelleCommuneEtablissement",
             "codePostalEtablissement",
             "codeCedexEtablissement",
             "libelleCedexEtablissement",
             "distributionSpecialeEtablissement",
             "libelleCommuneEtrangerEtablissement",
             "codePaysEtrangerEtablissement",
             "libellePaysEtrangerEtablissement",
             '_merge',
             'count'
             ]
def mergeInseeEtbs(df_insee, dataETBS, left_on, right_on, var_to_drop):
    """
    """
    
    data_merged_1 = (df_insee
                     .merge(
                         dataETBS,
                         how='right',
                         left_on=left_on,
                         right_on=right_on,
                         indicator=True)
                     # .compute()
                     )
    ### remove INSEE variables in data_merge_1
    unmatched_1 = (data_merged_1
             .loc[lambda x: x['_merge'].isin(["right_only"])]
             .drop(columns  = var_to_drop)
            )
    
    ### compute count merged:unmerged
    count_ = data_merged_1.groupby('_merge')["_merge"].count()
    
    dic_ = {
        'unmerged':unmatched_1, 
        'count_':count_
    }
    
    return dic_

## Step 1:  merge sur siren et code postal

### Processus
 
- Merge sur siren & codePostalEtablissement

Missing values:

- codePostalEtablissement:9950
- Code_Postal: 10092



In [ ]:
df_siren_to_find.shape

In [ ]:
m1 = mergeInseeEtbs(
    df_insee=subset_insee_count,
    dataETBS=df_siren_to_find,
    left_on=["siren", 'codePostalEtablissement'],
    right_on=['siren', 'Code_Postal'],
    var_to_drop=var_insee)

In [ ]:
m1['count_']

## Step 2:  merge sur siren et code commune

### Processus
 
- Merge sur siren & codeCommuneEtablissement

Missing values:

- codeCommuneEtablissement:168253
- Code_Commune: 99457

In [ ]:
m2 = mergeInseeEtbs(
    df_insee=subset_insee_count,
    dataETBS=m1['unmerged'],
    left_on=["siren", 'codeCommuneEtablissement'],
    right_on=['siren', 'Code_Commune'],
    var_to_drop=var_insee)

In [ ]:
m2['count_']

In [ ]:
(m2['count_'][1]/(5116526))  * 100

In [ ]:
m2['unmerged'].head()

### Step 3:  merge sur siren et libélé commune


On peut regarder le nombre de valeurs manquantes par champs pour les siren

Par exemple, dans le fichier non matché de l'inpi, il y a 49807 SIREN sans SIRET. Ses mêmes SIREN représententent 84760 observations dans le fichier de l'INSEE

In [ ]:
len(m2['unmerged'])

In [ ]:
siren_unique(
    df= subset_insee_count.loc[lambda x : x['siren'].isin(
m2['unmerged']['siren'].to_list())]
                               )

In [ ]:
subset_insee_count.loc[lambda x : x['siren'].isin(
m2['unmerged']['siren'].to_list())].isna().sum().sort_values()/ \
len(subset_insee_count.loc[lambda x : x['siren'].isin(
m2['unmerged']['siren'].to_list())])

In [ ]:
m2['unmerged'].isna().sum().sort_values()/len(m2['unmerged'])

Ajout matching des communes pour retrouver le libelé commune de l'INSEE

ATTENTION, il faut nétoyer la variables ville dans l'INSEE. Veuillez regarder le fichier `communes.xlsx` pour voir les différents problèmes

ex: 
- CEDEX, cedex, digit, (d+), 

attention, l'arrondissement peut être mis entre parenthèse 

- MARSEILLE (7E)

- process:
    - creer variables avec numeric seulement
    - recreer ville 2 si test pas NAN pour avoir l'arrondissement
    - virer les differentes informations dans ville via regex

In [ ]:
regex = 'CEDEX|cedex|Cedex|\([^)]*\)|/\s\s+/|^\d+\s|\s\d+\s|\s\d+$|\d+|\.|\--|COMMUNE DE |COMMUNE DE|commune de |commune de|Commune de |Commune de |\s$'
test_adress = m2['unmerged'].copy()
test_adress['test'] = m2['unmerged']['Ville'].str.extract(r'(\d+)')
test_adress['Ville_clean'] = test_adress['Ville'].str.replace(regex,'')
test_adress['Ville_clean'] = test_adress['Ville_clean'].str.replace('\s$|\s^','')
test_adress['ville2'] = np.where(
    np.logical_and(
         ~test_adress['test'].isin([np.nan]),
        test_adress['test'].str.len() <=2
    )
   ,
    test_adress['Ville_clean'] + '' + test_adress['test'].astype(str),
    test_adress['Ville_clean']
)

In [ ]:
test_adress[test_adress['Ville'].str.contains('VIRIEU LE GRAND', regex  = False,
                                                    na=False)]

On essaye sur ville, et ensuite on enlève les accents 

In [ ]:
temp = test_adress.merge(communes,
                         left_on='ville2',
                         right_on='possibilite',
                         how='left',
                         indicator=True)

temp1 = pd.concat([
    temp.loc[lambda x: x['_merge'].isin(['both'])],
    (temp
     .loc[lambda x: x['_merge'].isin(['left_only'])]
     .drop(columns=['ncc', 'possibilite', '_merge'])
     .merge(communes,
            left_on='Ville_clean',
            right_on='possibilite',
            how='left',
            indicator=True)
     )

])

temp1 = pd.concat([
    temp.loc[lambda x: x['_merge'].isin(['both'])],
    (temp
     .loc[lambda x: x['_merge'].isin(['left_only'])]
     .drop(columns=['ncc', 'possibilite', '_merge'])
     .assign(
         noaccent=lambda x: x['Ville_clean'].str.normalize('NFKD')
         .str.encode('ascii', errors='ignore')
         .str.decode('utf-8'))
     ).merge(communes,
             left_on='noaccent',
             right_on='possibilite',
             how='left',
             indicator=True)])
temp1.groupby('_merge')["_merge"].count()

In [ ]:
len(temp1)

### test unmerged

In [ ]:
var_insee = ['siret',
             "numeroVoieEtablissement",
             "indiceRepetitionEtablissement",
             "typeVoieEtablissement",
             "libelleVoieEtablissement",
             "complementAdresseEtablissement",
             "codeCommuneEtablissement",
             # "libelleCommuneEtablissement",
             "codePostalEtablissement",
             "codeCedexEtablissement",
             "libelleCedexEtablissement",
             "distributionSpecialeEtablissement",
             "libelleCommuneEtrangerEtablissement",
             "codePaysEtrangerEtablissement",
             "libellePaysEtrangerEtablissement",
             '_merge', 'count'
             ]

m3 = mergeInseeEtbs(
    df_insee=subset_insee_count,
    dataETBS=temp1.drop(columns=['possibilite',
                                 '_merge', 'noaccent',
                                 'Ville_clean',
                                 'ville2']),
    left_on=["siren", 'libelleCommuneEtablissement'],
    right_on=['siren', 'ncc'],
    var_to_drop=var_insee
)

In [ ]:
m3['count_']

In [ ]:
len(m3['unmerged'].loc[lambda x:x['Type'].isin(['SEP', 'PRI'])])

In [ ]:
siren_unique(
    df= subset_insee_count.loc[lambda x : x['siren'].isin(
m3['unmerged']['siren'].to_list())]
                               )

In [ ]:
#m3_bis = mergeInseeEtbs(
#    df_insee=subset_insee_count,
#    dataETBS= m3['unmerged'].loc[lambda x: ~x['test'].isin([np.nan])],
#    left_on=["siren", 'codePostalEtablissement'],
#    right_on=['siren', 'test'],
#    var_to_drop=var_insee
#)
#m3_bis['count_']

### Test entreprise un seul etb

On match uniquement lorsque le nombre de SIREN/SIRET est égal a 1. Il se peut que pour plusieurs raisons il y ai plusieurs SIREN dans l'INPI mais qu'un seul SIRET dans l'INSEE C'est du a a fermeture d'établissement Nous n'avons pas encore le véritable stock d'etb a l'INPI

In [ ]:
var_insee = [#'siret',
             "numeroVoieEtablissement",
             "indiceRepetitionEtablissement",
             "typeVoieEtablissement",
             "libelleVoieEtablissement",
             #"complementAdresseEtablissement",
             #"codeCommuneEtablissement",
             #"libelleCommuneEtablissement",
             #"codePostalEtablissement",
             "codeCedexEtablissement",
             "libelleCedexEtablissement",
             "distributionSpecialeEtablissement",
             "libelleCommuneEtrangerEtablissement",
             "codePaysEtrangerEtablissement",
             "libellePaysEtrangerEtablissement",
             #'_merge', 
    'count'
             ]

test_1 = (subset_insee_count
          .loc[lambda x : 
               x['siren'].isin(m3['unmerged']['siren'].to_list())
               & x['count'].isin([1])
              ]
          .merge(
              (m3['unmerged']
               .loc[lambda x:x['Type'].isin(['SEP', 'PRI'])]
              .drop(columns = 'libelleCommuneEtablissement')),
              on='siren',
              how='right',
              indicator=True
          )
          #.loc[lambda x: x['_merge'].isin(['both'])]
          #.drop(columns = var_insee)
          )  # .groupby('_merge')["_merge"].count()

Les right only correspondent aux SIREN qui ont plus d'un SIRET a l'INSEE. Nous nous focalisons uniqument sur les both, qui sont de véritables 1.

In [ ]:
test_1.groupby('_merge')["_merge"].count()

In [ ]:
subset_insee_count.loc[lambda x: x['siren'].isin(['388383374'])]

In [ ]:
test_1 = test_1.merge(
    (test_1
     .groupby('siren')['siren']
     .count()
     .rename('count')
     .reset_index())
)

In [ ]:
test_1['count'].value_counts()

In [ ]:
subset_insee_count.loc[lambda x: x['siren'].isin(['300011335'])]

In [ ]:
subset_insee_count.loc[lambda x: x['siren'].isin(['300018017'])]

In [ ]:
subset_insee_count.loc[lambda x: x['siren'].isin(['998239214'])]

On garde ceux ou on est "sur" qu'il y a qu'un seul SIREN. SI count > 1, c'est du a des fermetures que l'INSEE a purgé dans la base mais pas l'INPI

In [ ]:
siren_count_1_found = test_1.loc[lambda x: x['count'].isin([1])]['siren'].to_list()

### Match avec adresse

On selectionne uniquement ceux pas matché.
Pour accélerer la recherche, on utilise que le sous ensemble de siren a vérifier dans le fichier INSEE

Verifier si on peut matcher avec le numéro de l'adresse -> au cas ou principal et secondaire dans le même endroit

Il faut retravailler les adresses:

- Upper case

La recherche se fait sur le libellé adresse. Dans l'INSEE, pas de numéro de voie, ni de typologie (rue, avenue, etc)

ALL: Allée
AV: Avenue
BD: Boulevard
CAR: Carrefour
CHE: Chemin
CHS: Chaussée
CITE: Cité
COR: Corniche
CRS: Cours
DOM: Domaine
DSC: Descente
ECA: Ecart
ESP: Esplanade
FG: Faubourg
GR: Grande Rue
HAM: Hameau
HLE: Halle
IMP: Impasse
LD: Lieu dit
LOT: Lotissement
MAR: Marché
MTE: Montée
PAS: Passage
PL: Place
PLN: Plaine
PLT: Plateau
PRO: Promenade
PRV: Parvis
QUA: Quartier
QUAI: Quai
RES: Résidence
RLE: Ruelle
ROC: Rocade
RPT: Rond Point
RTE: Route
RUE: Rue
SEN: Sente - Sentier
SQ: Square
TPL: Terre-plein
TRA: Traverse
VLA: Villa
VLGE: Village

In [ ]:
additional = ["Avenue",
"Boulevard",
"Carrefour",
"Chemin",
"Chaussee",
"Cite",
"Corniche",
"Cours",
"Domaine",
"Descente",
"Ecart",
"Esplanade",
"Faubourg",
"Grande Rue",
"Hameau",
"Halle",
"Impasse",
"Lieu dit",
"Lotissement",
"Marche",
"Montee",
"Passage",
"Place",
"Plaine",
"Plateau",
"Promenade",
"Parvis",
"Quartier",
"Quai",
"Residence",
"Ruelle",
"Rocade",
"Rond Point",
"Route",
"Rue",
"Sentier",
"Square",
"Terre plein",
"Traverse",
"Villa",
"Village",
'bp', 'cedex']

In [ ]:
import nltk
#nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop_words = stopwords.words('french')
stop_words.extend(additional)
upper_stop = [i.upper() for i in stop_words]

In [ ]:
def create_split_adress(x):
    """
    """
    split_ = x.str.split().to_list()
    
    #split_ = ''.join(str(e) for e in split_)
    #reg = '|'.join(split_)
    return  split_


def create_regex_adress(x):
    """
    """
    try:
        split_ = ["^" + i + "$" for i in x]
        reg = '|'.join(split_)
    except:
        reg = np.nan
    return  reg

def lookupInseeInpi(siren, regex_):
    """
    """
    try:
        siret_ = sous_ensemble_insee.loc[lambda x: 
                                  x['siren'].isin([siren])
                      & x['libelleVoieEtablissement'].str.contains(
                          regex_, 
                          case = False, 
                          regex = True)
                      ]['siret']
        return siret_.values[0]
    except:
        return np.nan 

In [ ]:
temp_adresse = m3['unmerged'].loc[lambda x: ~x['siren'].isin(siren_count_1_found)].copy()
sous_ensemble_insee = subset_insee_count.loc[lambda x: x['siren'].isin(temp_adresse['siren'].to_list())]
len(sous_ensemble_insee)

temp_adresse = temp_adresse.assign(
    Adresse_Ligne1_clean=lambda x: x['Adresse_Ligne1'].str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
    .str.replace('[^\w\s]|\d+', '')
    .str.upper(),
    Adresse_Ligne2_clean=lambda x: x['Adresse_Ligne2'].str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
    .str.replace('[^\w\s]|\d+', '')
    .str.upper(),
    Adresse_Ligne3_clean=lambda x: x['Adresse_Ligne3'].str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
    .str.replace('[^\w\s]|\d+', '')
    .str.upper()
)
temp_adresse['Adresse_Ligne1_clean'] = (temp_adresse['Adresse_Ligne1_clean']
 .apply(lambda x: ' '.join([word for word in str(x).split() if word not in (upper_stop)]))
 )

temp_adresse['Adresse_Ligne2_clean'] = (temp_adresse['Adresse_Ligne2_clean']
 .apply(lambda x: ' '.join([word for word in str(x).split() if word not in (upper_stop)]))
 )

temp_adresse['Adresse_Ligne3_clean'] = (temp_adresse['Adresse_Ligne3_clean']
 .apply(lambda x: ' '.join([word for word in str(x).split() if word not in (upper_stop)]))
 )

temp_adresse = temp_adresse.assign(
    test = lambda x : create_split_adress(x['Adresse_Ligne1_clean']),
    test2 = lambda x : create_split_adress(x['Adresse_Ligne2_clean']),
    test3 = lambda x : create_split_adress(x['Adresse_Ligne3_clean'])
)

temp_adresse['test_1'] = temp_adresse['test'].apply(lambda x: create_regex_adress(x))
temp_adresse['test_2'] = temp_adresse['test2'].apply(lambda x: create_regex_adress(x))
temp_adresse['test_3'] = temp_adresse['test3'].apply(lambda x: create_regex_adress(x))

In [ ]:


temp_adresse.loc[lambda x: x['siren'].isin(['343304465'])]

In [ ]:
temp_adresse.head(1)

In [ ]:
sous_ensemble_insee.loc[lambda x: x['siren'].isin([temp_adresse.iloc[0, 0]])]

In [ ]:
lookupInseeInpi(siren = temp_adresse.iloc[0, 0],
                regex_ = temp_adresse.iloc[0, -2])

Pour accelerer le code, on utilise uniquement les adresses sans les na

#### test_1

In [ ]:
test_1 = temp_adresse.loc[lambda x: ~x['test_1'].isin(['nan'])]
test_1.head()

In [ ]:
test_1['Type'].value_counts()

In [ ]:
%%time
test_1.head(10).apply(lambda x: lookupInseeInpi(
    siren = x['siren'],
    regex_ = x['test_1']),
    axis = 1 )

In [ ]:
lookupInseeInpi(siren = test_1.iloc[2, 0],
                regex_ = test_1.iloc[2, -2])

In [ ]:
sous_ensemble_insee.loc[lambda x: x['siren'].isin([test_1.iloc[2, 0]])]

In [ ]:
((17000 * 2.62) /10)/60

In [ ]:
%%time
test_1['siret_1'] = test_1.apply(lambda x: lookupInseeInpi(
    siren = x['siren'],
    regex_ = x['test_1']),
    axis = 1 )

In [ ]:
test_1.shape

In [ ]:
test_1.loc[lambda x : ~x['siret_1'].isin([np.nan])].head()

In [ ]:
test_1.loc[lambda x : ~x['siret_1'].isin([np.nan])].shape

In [ ]:
(sous_ensemble_insee
 .loc[lambda x: x['siren'].isin(["395395759"])]
)

In [ ]:
test_1['siret_1'].isna().sum()

#### test_2

In [ ]:
%%time
test_2 = temp_adresse.loc[lambda x: ~x['test_2'].isin(['nan'])]
test_2['siret_2'] = test_1.apply(lambda x: lookupInseeInpi(
    siren = x['siren'],
    regex_ = x['test_2']),
    axis = 1 )

In [ ]:
test_2['siret_2'].isna().sum()

In [ ]:
test_2.shape

In [ ]:
test_2.loc[lambda x : ~x['siret_2'].isin([np.nan])].head()

In [ ]:
test_2.loc[lambda x : ~x['siret_2'].isin([np.nan])].shape

In [ ]:
(sous_ensemble_insee
 .loc[lambda x: x['siren'].isin(["394674881"])]
)

#### test_3

In [ ]:
%%time
test_3 = temp_adresse.loc[lambda x: ~x['test_3'].isin(['nan'])]
test_3['siret_3'] = test_1.apply(lambda x: lookupInseeInpi(
    siren = x['siren'],
    regex_ = x['test_3']),
    axis = 1 )

In [ ]:
test_3.loc[lambda x : ~x['siret_3'].isin([np.nan])].shape

In [ ]:
test_3.loc[lambda x : ~x['siret_3'].isin([np.nan])].head()

In [ ]:
(sous_ensemble_insee
 .loc[lambda x: x['siren'].isin(["302556832"])]
)

In [ ]:
lookupInseeInpi(siren = '302556832',
                regex_ = '^AVE$|^MICHEL$|^JOURDAN$')

temp1.head(

In [ ]:
import os
os.remove('communes.xlsx')
temp1.loc[lambda x: x['_merge'].isin(['left_only'])].drop_duplicates('ville2').to_excel('communes.xlsx')

In [ ]:
temp.assign(url = lambda x :
            'https://data.inpi.fr/entreprises/' + x["Siren"] )